<a href="https://colab.research.google.com/github/akhaire21/Semi-Supervised-Ensemble-Model/blob/main/Microglia_AD_Research_Data_Preprocessing_FinalVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read DeepGWAS Output
21,016,305 records with predicted probabilities

In [ ]:
# Connect to Google Drive

from google.colab import drive
drive = drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Load DeepGWAS Output and identify ~47K possible AD variants identified by DEEPGWAS Model
import pandas as pd
df_Deep = pd.read_csv('/content/drive/My Drive/Regeneron_STS_2023/Model/Input_Data/DeepGWAS_Output.txt',delimiter='\t',
                      usecols = ['log10p','prob'], dtype={'SNPID': float, 'prob': float})
df_Deep['binned'] = [1 if x > 0.5 else 0 for x in df_Deep['prob']]
# 21016305 rows

In [ ]:
import math
p = -(math.log10(5e-08))
df_Deep['pval_flg'] = [1 if x > p else 0 for x in df_Deep['log10p']]

In [ ]:
df_Deep.columns

Index(['log10p', 'prob', 'binned', 'pval_flg'], dtype='object')

In [ ]:
df_Deep[['binned','pval_flg']].value_counts()

binned  pval_flg
0       0           20967861
        1              47733
1       1                702
        0                  9
dtype: int64

In [ ]:
# Load original DeepGWAS input and get additional columns needed for processing
df = pd.read_csv("/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/FINAL_Full33Columns_start_end.txt", header=0, delimiter = '\t',
                  usecols = ['SNPID','chromosome', 'start', 'end', 'RSID', 'pvalue.x'], 
                  dtype={'SNPID': str})

In [ ]:
df.head()
# 21016305 rows

,SNPID,chromosome,start,end,RSID,pvalue.x
0,0,chr1,229820,229821,rs1159884564,0.2303
1,chr1_529825_CT,chr1,529825,529826,rs888953847,0.9626
2,0,chr1,531142,531143,rs1040232850,0.2670
3,chr1_566327_AG,chr1,566327,566328,rs1390538076,0.8214
4,chr1_581537_AG,chr1,581537,581538,rs1250812823,0.3411


In [ ]:
result = pd.concat([df, df_Deep.reindex(df.index)], axis=1)
result.shape
#(21016305, 9)

(21016305, 10)

In [ ]:
del df
del df_Deep

In [ ]:
df = result[(result['binned'] == 0) & (result['pval_flg'] == 1)].copy()
#47333x10

In [ ]:
df[df.index.duplicated()]

,SNPID,chromosome,start,end,RSID,pvalue.x,log10p,prob,binned,pval_flg


In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.head()

,SNPID,chromosome,start,end,RSID,pvalue.x,log10p,prob,binned,pval_flg
0,chr1_916549_AG,chr1,916549,916550,rs6660139,0.000647,7.343164,0.0,0,1
1,chr1_961827_AG,chr1,961827,961828,rs3121556,0.000137,8.897722,0.0,0,1
2,chr1_963249_CT,chr1,963249,963250,rs2710870,0.000090,9.311266,0.0,0,1
3,chr1_971224_AG,chr1,971224,971225,rs2799055,0.000223,8.408339,0.0,0,1
4,chr1_972134_CT,chr1,972134,972135,rs3121575,0.000568,7.473565,0.0,0,1


In [ ]:
del result

In [ ]:
## Get Allele to use in LEFT JOIN with Glass/Shen LAB Data
df_Bell = pd.read_csv("/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/Bellenguez_21_BedLifted_IdAllelesIncluded.bed", header=0, delimiter = '\t')
df_Bell.columns=["chr", "start", "end", "p_value", "RSID",	"effect_allele",	"other_allele"]
df_Bell.head()
#21061705 x 7 rows

,chr,start,end,p_value,RSID,effect_allele,other_allele
0,chr1,531142,531143,0.2670,rs1040232850,CTG,C
1,chr1,566327,566328,0.8214,rs1390538076,A,G
2,chr1,581537,581538,0.3411,rs1250812823,A,G
3,chr1,661906,661907,0.3542,rs991450070,C,G
4,chr1,662613,662614,0.6424,rs151190501,A,G


In [ ]:
df = pd.merge(df, df_Bell, how='left', left_on=['chromosome','start','RSID','pvalue.x'], right_on=['chr','start','RSID','p_value'])
del df_Bell

In [ ]:
df.head()
# 47733 records

,SNPID,chromosome,start,end_x,RSID,pvalue.x,log10p,prob,binned,pval_flg,chr,end_y,p_value,effect_allele,other_allele
0,chr1_916549_AG,chr1,916549,916550,rs6660139,0.000647,7.343164,0.0,0,1,chr1,916550,0.000647,A,G
1,chr1_961827_AG,chr1,961827,961828,rs3121556,0.000137,8.897722,0.0,0,1,chr1,961828,0.000137,A,G
2,chr1_963249_CT,chr1,963249,963250,rs2710870,0.000090,9.311266,0.0,0,1,chr1,963250,0.000090,T,C
3,chr1_971224_AG,chr1,971224,971225,rs2799055,0.000223,8.408339,0.0,0,1,chr1,971225,0.000223,A,G
4,chr1_972134_CT,chr1,972134,972135,rs3121575,0.000568,7.473565,0.0,0,1,chr1,972135,0.000568,T,C


In [ ]:
df[df.duplicated(['RSID'], keep=False)]

,SNPID,chromosome,start,end_x,RSID,pvalue.x,log10p,prob,binned,pval_flg,chr,end_y,p_value,effect_allele,other_allele


In [ ]:
df = df[['chromosome','start','end_x','RSID','effect_allele','other_allele','prob','binned']].copy()

In [ ]:
df['chromosome'].value_counts()

chr6                     5806
chr17                    4671
chr2                     3699
chr19                    3186
chr1                     2985
chr11                    2932
chr7                     2839
chr5                     2576
chr10                    2348
chr8                     2064
chr3                     1975
chr4                     1945
chr16                    1826
chr14                    1731
chr15                    1696
chr12                    1419
chr9                     1062
chr20                     830
chr13                     624
chr18                     568
chr21                     493
chr22                     455
chr17_gl000204_random       2
chr1_gl000192_random        1
Name: chromosome, dtype: int64

In [ ]:
## Drop the incorrect records:
## chr17_gl000204_random       2
## chr1_gl000192_random        1
df =df[df['chromosome'] != 'chr17_gl000204_random']
df =df[df['chromosome'] != 'chr1_gl000192_random']

In [ ]:
df['chromosome'].value_counts()
#47730 records

chr6     5806
chr17    4671
chr2     3699
chr19    3186
chr1     2985
chr11    2932
chr7     2839
chr5     2576
chr10    2348
chr8     2064
chr3     1975
chr4     1945
chr16    1826
chr14    1731
chr15    1696
chr12    1419
chr9     1062
chr20     830
chr13     624
chr18     568
chr21     493
chr22     455
Name: chromosome, dtype: int64

In [ ]:
df.rename(columns={'chromosome': 'chr', 'end_x': 'end'}, inplace=True)

In [ ]:
df.head()

,chr,start,end,RSID,effect_allele,other_allele,prob,binned
0,chr1,916549,916550,rs6660139,A,G,0.0,0
1,chr1,961827,961828,rs3121556,A,G,0.0,0
2,chr1,963249,963250,rs2710870,T,C,0.0,0
3,chr1,971224,971225,rs2799055,A,G,0.0,0
4,chr1,972134,972135,rs3121575,T,C,0.0,0


# Update True Positive & Negative Labels

In [ ]:
# Known Alzheimer's Variants - Positive & Negative Labels
# https://www.science.org/doi/10.1126/science.abi8654
# https://www.biorxiv.org/content/biorxiv/early/2021/06/15/2021.06.14.448395.full.pdf
#df_AD_CRISPR = pd.read_csv('/content/drive/My Drive/AD_CRISPR.tsv',header=0, delimiter='\t')
df_AD_P_N = pd.read_csv('/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/AD_Positive_Negative_Labels.tsv',header=0, delimiter='\t')
#df_AD_CRISPR['flg'] = 1
#df_AD_Loci.columns = ['chr','start','flg']
df_AD_P_N.head(10)

,chr,start,rsid,flg,comment
0,chr10,11720308.0,rs7920721,2,CRISPR
1,chr14,53333616.0,rs7144029,2,CRISPR
2,chr14,53391680.0,rs17125924,2,CRISPR
3,chr16,19722366.0,rs3829539,2,CRISPR
4,chr16,19729016.0,rs4782272,2,CRISPR
5,chr16,19749709.0,rs13331873,2,CRISPR
6,chr19,45241638.0,rs2927437,2,CRISPR
7,chr19,45354296.0,rs11666329,2,CRISPR
8,chr19,45379516.0,rs412776,2,CRISPR
9,chr19,45387459.0,rs12972156,2,CRISPR


In [ ]:
df = pd.merge(df, df_AD_P_N, on=['chr','start'],how='left')
df['flg'] = df['flg'].fillna(-1)
df.loc[df['flg'] == 1, 'Target'] = 1
df.loc[df['flg'] == 2, 'Target'] = 2
df.loc[df['flg'] == -1, 'Target'] = -1
df[df['Target']==1]

,chr,start,end,RSID,effect_allele,other_allele,prob,binned,rsid,flg,comment,Target
2319,chr1,207786289,207786290,rs6701713,A,G,0.0,0,rs6701713,1.0,CRISPR,1.0
5880,chr11,47568344,47568345,rs10838726,C,G,0.0,0,rs10838726,1.0,CRISPR,1.0
6508,chr11,59936979,59936980,rs667897,A,G,0.0,0,rs667897,1.0,CRISPR,1.0
6608,chr11,60019150,60019151,rs636317,T,C,0.0,0,rs636317,1.0,CRISPR,1.0
11307,chr14,92932437,92932438,rs4900124,A,G,0.0,0,rs4900124,1.0,Table_7a7d,1.0
11309,chr14,92932479,92932480,rs4904922,T,G,0.0,0,rs4904922,1.0,Table_7a7d,1.0
11310,chr14,92932485,92932486,rs4904923,C,G,0.0,0,rs4904923,1.0,Table_7a7d,1.0
11311,chr14,92932575,92932576,rs10130373,A,C,0.0,0,rs10130373,1.0,Table_7a7d,1.0
11352,chr14,92938382,92938383,rs36026988,T,C,0.0,0,rs36026988,1.0,CRISPR,1.0
13909,chr16,19710702,19710703,rs9935063,A,G,0.0,0,rs9935063,1.0,CRISPR,1.0


In [ ]:
df.drop(df.columns[8], axis=1,inplace=True)
df.drop('flg', axis=1,inplace=True)
df.drop('comment', axis=1,inplace=True)

In [ ]:
df.head()

,chr,start,end,RSID,effect_allele,other_allele,prob,binned,Target
0,chr1,916549,916550,rs6660139,A,G,0.0,0,-1.0
1,chr1,961827,961828,rs3121556,A,G,0.0,0,-1.0
2,chr1,963249,963250,rs2710870,T,C,0.0,0,-1.0
3,chr1,971224,971225,rs2799055,A,G,0.0,0,-1.0
4,chr1,972134,972135,rs3121575,T,C,0.0,0,-1.0


In [ ]:
df['prob'] = df['prob'].fillna(0)

In [ ]:
print('Target Value Distribution:')
print(df['Target'].value_counts())

Target Value Distribution:
-1.0    47702
 1.0       14
 2.0       14
Name: Target, dtype: int64


# Data Analysis and cleanup

In [ ]:
print(df.columns)

Index(['chr', 'start', 'end', 'RSID', 'effect_allele', 'other_allele', 'prob',
       'binned', 'Target'],
      dtype='object')


In [ ]:
# General information about the dataframe
print(df.info())
# 47730 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47730 entries, 0 to 47729
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   chr            47730 non-null  object 
 1   start          47730 non-null  int64  
 2   end            47730 non-null  int64  
 3   RSID           47730 non-null  object 
 4   effect_allele  47730 non-null  object 
 5   other_allele   47730 non-null  object 
 6   prob           47730 non-null  float64
 7   binned         47730 non-null  int64  
 8   Target         47730 non-null  float64
dtypes: float64(2), int64(3), object(4)
memory usage: 3.6+ MB
None


In [ ]:
## Write df to google drive
df.to_csv('/content/drive/My Drive/Regeneron_STS_2023/Model/Input_Data/SSL_PUNU_Model_Input_IntermediateTMP.txt', sep='\t',index=False)

In [ ]:
df = df[['chr','start','end','Target']].copy()
df = df.drop(df[df['chr'] == '0'].index)
df = df.sort_values(by=['chr', 'start', 'end'])
df.head()

,chr,start,end,Target
0,chr1,916549,916550,-1.0
1,chr1,961827,961828,-1.0
2,chr1,963249,963250,-1.0
3,chr1,971224,971225,-1.0
4,chr1,972134,972135,-1.0


# ShenLab: TPM : xc119 and xc120

In [ ]:
df_TPM = pd.read_csv('/content/drive/My Drive/rnaseq_TPM.txt', header=0,delimiter='\t') 
df_TPM = df_TPM.iloc[:,[0,1,2,17,18]].copy()
df_TPM.columns = ['chr','start','end','xc119','xc120']

In [ ]:
df_TPM.head()

,chr,start,end,xc119,xc120
0,chr1,903641,927394,0.00,0.00
1,chr1,903641,927394,0.00,0.00
2,chr1,927395,936954,0.02,0.00
3,chr1,927395,936954,4.21,2.84
4,chr1,936955,942417,0.00,0.00


In [ ]:
df_TPM = df_TPM.set_index(['chr'])

In [ ]:
df_TPM.isnull().values.any()

False

In [ ]:
from tqdm import tqdm
import numpy as np


ls = []
i=0
for row in tqdm(df.itertuples()):
  c=row.chr
  s=row.start
  e=row.end
  T=row.Target

  if c in df_TPM.index:
    df_sub = df_TPM.loc[c]
    xc119 = df_sub[(df_sub['start'] <= s) & (df_sub['end'] >= e)]['xc119']
    xc120 = df_sub[(df_sub['start'] <= s) & (df_sub['end'] >= e)]['xc120']

    if len(xc119) > 0:
      vv = np.average(xc119.values)
      xc119_v = vv
    else:
      xc119_v = 0.0

    if len(xc120) > 0:
      vv = np.average(xc120.values)
      xc120_v = vv
    else:
      xc120_v = 0.0

    aa = [c,s,e,T,xc119_v,xc120_v]
    ls.append(aa)

    i+=1
    if i%5000 == 0:
      print(i)
  else:
    pass

5063it [00:11, 476.10it/s]

5000


10063it [00:21, 478.02it/s]

10000


15085it [00:32, 498.83it/s]

...

40066it [01:25, 459.38it/s]

40000


45046it [01:35, 452.37it/s]

45000


47730it [01:41, 471.46it/s]


In [ ]:
hm = pd.DataFrame(ls)
hm.columns= ['chr', 'start', 'end', 'Target', 'xc119_TPM', 'xc120_TPM']
hm[hm['xc119_TPM'] > 0]

,chr,start,end,Target,xc119_TPM,xc120_TPM
1,chr1,961827,961828,-1.0,20.370000,18.240000
2,chr1,963249,963250,-1.0,20.370000,18.240000
3,chr1,971224,971225,-1.0,20.370000,18.240000
4,chr1,972134,972135,-1.0,20.370000,18.240000
5,chr1,973336,973337,-1.0,20.370000,18.240000
...,...,...,...,...,...,...
47718,chr9,140038802,140038803,-1.0,0.010000,0.040000
47720,chr9,140472913,140472914,-1.0,11.220000,8.690000
47721,chr9,140488857,140488858,-1.0,22.540000,17.180000
47722,chr9,140500443,140500444,-1.0,14.016667,10.953333


In [ ]:
df = hm.copy()
del df_TPM
del hm

# GlassLab2019: PLAC : Count and Expected

In [ ]:
df_PLAC = pd.read_csv('/content/drive/My Drive/plac_seq.txt', delimiter='\t', 
                      low_memory=False) 
df_PLAC = df_PLAC.iloc[:,[0,1,2,6,7]].copy()
df_PLAC.columns = ['chr','start','end','cnt','expected']

In [ ]:
df_PLAC = df_PLAC.set_index(['chr'])

In [ ]:
df_PLAC.isnull().values.any()

False

In [ ]:
from tqdm import tqdm

ls = []

i=0
for row in tqdm(df.itertuples()):
  c=row.chr
  s=row.start
  e=row.end
  T=row.Target
  x119=row.xc119_TPM
  x120=row.xc120_TPM

  df_sub = df_PLAC.loc[c]
  cnt = df_sub[(df_sub['start'] <= s) & (df_sub['end'] >= e)]['cnt']
  expected = df_sub[(df_sub['start'] <= s) & (df_sub['end'] >= e)]['expected']

  if len(cnt) > 0:
    vv = np.average(cnt.values)
    cnt_v = vv
  else:
    cnt_v = 0.0

  if len(expected) > 0:
    vv = np.average(expected.values)
    expected_v = vv
  else:
    expected_v = 0.0

  aa = [c,s,e,T,x119,x120,cnt_v,expected_v]
  ls.append(aa)

  i+=1
  if i%5000 == 0:
    print(i)

5040it [00:15, 339.54it/s]

5000


10064it [00:31, 342.73it/s]

10000


15048it [00:46, 319.70it/s]

...

40034it [02:02, 309.70it/s]

40000


45052it [02:19, 305.85it/s]

45000


47730it [02:28, 321.99it/s]


In [ ]:
hm = pd.DataFrame(ls)
hm.columns= ['chr', 'start', 'end', 'Target', 'xc119_TPM', 'xc120_TPM', 'cnt', 'expected']
hm[hm['expected'] > 0]

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected
1,chr1,961827,961828,-1.0,20.37,18.24,11.000000,2.431481
2,chr1,963249,963250,-1.0,20.37,18.24,11.000000,2.431481
41,chr1,1932309,1932310,-1.0,0.00,0.00,53.000000,22.687543
64,chr1,6644723,6644724,-1.0,43.18,29.52,53.250000,22.569102
65,chr1,6659505,6659506,-1.0,0.00,0.00,18.000000,6.825826
...,...,...,...,...,...,...,...,...
47694,chr9,134228216,134228217,-1.0,0.00,0.00,12.714286,3.455096
47695,chr9,134228280,134228281,-1.0,0.00,0.00,12.714286,3.455096
47696,chr9,134228471,134228472,-1.0,0.00,0.00,12.714286,3.455096
47697,chr9,134228914,134228915,-1.0,0.00,0.00,12.714286,3.455096


In [ ]:
df = hm.copy()
del df_PLAC
del hm

# ShenLab: pcHi: Nreads and Score

In [ ]:
df_pcHi = pd.read_csv('/content/drive/My Drive/pcHi_C.txt', header=0,delimiter='\t') 
df_pcHi = df_pcHi.iloc[:,[0,1,2,8,9]].copy()
df_pcHi.columns = ['chr','start','end','Nreads','score']

In [ ]:
df_pcHi = df_pcHi.set_index(['chr'])

In [ ]:
df_pcHi.isnull().values.any()

False

In [ ]:
from tqdm import tqdm

ls = []

i=0
for row in tqdm(df.itertuples()):
  c=row.chr
  s=row.start
  e=row.end
  T=row.Target
  x119=row.xc119_TPM
  x120=row.xc120_TPM
  cnt=row.cnt
  exp=row.expected

  df_sub = df_pcHi.loc[c]
  Nreads = df_sub[(df_sub['start'] <= s) & (df_sub['end'] >= e)]['Nreads']
  score = df_sub[(df_sub['start'] <= s) & (df_sub['end'] >= e)]['score']

  if len(Nreads) > 0:
    vv = np.average(Nreads.values)
    Nreads_v = vv
  else:
    Nreads_v = 0.0

  if len(score) > 0:
    vv = np.average(score.values)
    score_v = vv
  else:
    score_v = 0.0

  aa = [c,s,e,T,x119,x120,cnt,exp,Nreads_v,score_v]
  ls.append(aa)

  i+=1
  if i%5000 == 0:
    print(i)

5137it [00:06, 778.07it/s]

5000


10089it [00:13, 618.77it/s]

10000


15074it [00:20, 422.11it/s]

...

40159it [00:52, 788.50it/s]

40000


45104it [00:59, 772.05it/s]

45000


47730it [01:02, 758.73it/s]


In [ ]:
hm = pd.DataFrame(ls)
hm.columns= ['chr', 'start', 'end', 'Target', 'xc119_TPM', 'xc120_TPM', 'cnt',
             'expected', 'Nreads', 'score']
hm[hm['score'] > 0]

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected,Nreads,score
1,chr1,961827,961828,-1.0,20.37,18.24,11.0,2.431481,6.8,8.670000
2,chr1,963249,963250,-1.0,20.37,18.24,11.0,2.431481,6.8,8.670000
3,chr1,971224,971225,-1.0,20.37,18.24,0.0,0.000000,6.8,8.670000
4,chr1,972134,972135,-1.0,20.37,18.24,0.0,0.000000,6.8,8.670000
5,chr1,973336,973337,-1.0,20.37,18.24,0.0,0.000000,6.8,8.670000
...,...,...,...,...,...,...,...,...,...,...
47717,chr9,139922102,139922103,-1.0,3.62,3.12,0.0,0.000000,10.5,8.980000
47718,chr9,140038802,140038803,-1.0,0.01,0.04,0.0,0.000000,8.0,7.772857
47719,chr9,140055905,140055906,-1.0,0.00,0.00,0.0,0.000000,11.0,5.430000
47720,chr9,140472913,140472914,-1.0,11.22,8.69,0.0,0.000000,19.5,6.205000


In [ ]:
df = hm.copy()
del df_pcHi
del hm

# ATAC_wtcMG Full File: Value

In [ ]:
df_atac_wtcMG = pd.read_csv('/content/drive/My Drive/wtcMG_atac_ctrl_100bp.bedGraph', header=None,delimiter='\t') 
df_atac_wtcMG = df_atac_wtcMG.iloc[:,[0,1,2,3]].copy()
df_atac_wtcMG.columns = ['chr','start','end', 'value']

In [ ]:
df_atac_wtcMG = df_atac_wtcMG.sort_values(by=['chr', 'start', 'end'])
df_atac_wtcMG = df_atac_wtcMG.set_index(['chr'])

In [ ]:
df_atac_wtcMG.isnull().values.any()

False

In [ ]:
from tqdm import tqdm

ATAC_wtcMG = pd.DataFrame()
ls = []

i=0
for row in tqdm(df.itertuples()):
  c=row.chr
  s=row.start
  e=row.end
  T=row.Target
  x119=row.xc119_TPM
  x120=row.xc120_TPM
  cnt=row.cnt
  exp=row.expected
  N=row.Nreads
  sc=row.score

  df_sub = df_atac_wtcMG.loc[c]
  value = df_sub[(df_sub['start'] <= s) & (df_sub['end'] >= e)]['value']

  if len(value) > 0:
    vv = np.average(value.values)
    value_v = vv
  else:
    value_v = 0.0

  aa = [c,s,e,T,x119,x120,cnt,exp,N,sc,value_v]
  ls.append(aa)

  i+=1
  if i%5000 == 0:
    print(i)

5030it [00:44, 157.71it/s]

5000


10017it [01:19, 168.54it/s]

10000


15041it [01:44, 200.37it/s]

...

40022it [04:27, 129.00it/s]

40000


45023it [05:04, 148.29it/s]

45000


47730it [05:21, 148.44it/s]


In [ ]:
hm = pd.DataFrame(ls)
hm.columns= ['chr', 'start', 'end', 'Target', 'xc119_TPM', 'xc120_TPM', 'cnt',
             'expected', 'Nreads', 'score', 'score_ATAC_wtcMG']
hm[hm['score_ATAC_wtcMG'] > 0]

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected,Nreads,score,score_ATAC_wtcMG
0,chr1,916549,916550,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.066223
1,chr1,961827,961828,-1.0,20.37,18.24,11.0,2.431481,6.8,8.67,0.110371
2,chr1,963249,963250,-1.0,20.37,18.24,11.0,2.431481,6.8,8.67,0.066223
3,chr1,971224,971225,-1.0,20.37,18.24,0.0,0.000000,6.8,8.67,0.154519
4,chr1,972134,972135,-1.0,20.37,18.24,0.0,0.000000,6.8,8.67,0.033111
...,...,...,...,...,...,...,...,...,...,...,...
47724,chr9,140546015,140546016,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.022074
47725,chr9,140576488,140576489,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.022074
47726,chr9,140578044,140578045,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.011037
47728,chr9,140605202,140605203,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.011037


In [ ]:
df = hm.copy()
del df_atac_wtcMG
del hm

# GlassLab2017: CHIP Invitro: Full File : Value

In [ ]:
df_chip_invitro = pd.read_csv('/content/drive/My Drive/SRR5955098.bedGraph', header=None,delimiter='\t') 
df_chip_invitro = df_chip_invitro.iloc[:,[0,1,2,3]].copy()
df_chip_invitro.columns = ['chr','start', 'end', 'value']

In [ ]:
df_chip_invitro = df_chip_invitro.sort_values(by=['chr', 'start', 'end'])
df_chip_invitro = df_chip_invitro.set_index(['chr'])

In [ ]:
df_chip_invitro.isnull().values.any()

False

In [ ]:
from tqdm import tqdm

ls = []

i=0
for row in tqdm(df.itertuples()):
  c=row.chr
  s=row.start
  e=row.end
  T=row.Target
  x119=row.xc119_TPM
  x120=row.xc120_TPM
  cnt=row.cnt
  exp=row.expected
  N=row.Nreads
  sc=row.score
  scwtc=row.score_ATAC_wtcMG

  df_sub = df_chip_invitro.loc[c]
  value = df_sub[(df_sub['start'] <= s) & (df_sub['end'] >= e)]['value']

  if len(value) > 0:
    vv = np.average(value.values)
    value_v = vv
  else:
    value_v=0.0

  aa = [c,s,e,T,x119,x120,cnt,exp,N,sc,scwtc,value_v]
  ls.append(aa)

  i+=1
  if i%5000 == 0:
    print(i)

5077it [00:10, 632.87it/s]

5000


10123it [00:17, 742.60it/s]

10000


15119it [00:24, 878.36it/s]

...

40066it [00:59, 631.37it/s]

40000


45139it [01:11, 726.77it/s]

45000


47730it [01:14, 639.49it/s]


In [ ]:
hm = pd.DataFrame(ls)
hm.columns= ['chr', 'start', 'end', 'Target', 'xc119_TPM', 'xc120_TPM', 'cnt',
             'expected', 'Nreads', 'score', 'score_ATAC_wtcMG','score_chip_invitro']
hm[hm['score_chip_invitro'] > 0]

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected,Nreads,score,score_ATAC_wtcMG,score_chip_invitro
2,chr1,963249,963250,-1.0,20.370000,18.240000,11.0,2.431481,6.8,8.670,0.066223,0.030430
4,chr1,972134,972135,-1.0,20.370000,18.240000,0.0,0.000000,6.8,8.670,0.033111,0.030430
9,chr1,974180,974181,-1.0,20.370000,18.240000,0.0,0.000000,6.8,8.670,0.364224,0.121721
10,chr1,974199,974200,-1.0,20.370000,18.240000,0.0,0.000000,6.8,8.670,0.364224,0.121721
15,chr1,974494,974495,-1.0,20.370000,18.240000,0.0,0.000000,0.0,0.000,0.242816,0.152151
...,...,...,...,...,...,...,...,...,...,...,...,...
47714,chr9,139332657,139332658,-1.0,4.320000,4.230000,0.0,0.000000,13.0,7.675,0.033111,0.030430
47717,chr9,139922102,139922103,-1.0,3.620000,3.120000,0.0,0.000000,10.5,8.980,0.022074,0.395593
47720,chr9,140472913,140472914,-1.0,11.220000,8.690000,0.0,0.000000,19.5,6.205,0.165556,1.004200
47722,chr9,140500443,140500444,-1.0,14.016667,10.953333,0.0,0.000000,0.0,0.000,0.474595,0.182581


In [ ]:
df = hm.copy()
del df_chip_invitro
del hm

# GlassLab2017: CHIP Exvivo: Full File

In [ ]:
df_chip_exvivo = pd.read_csv('/content/drive/My Drive/SRR5955095.bedGraph', header=None,delimiter='\t') 
df_chip_exvivo = df_chip_exvivo.iloc[:,[0,1,2,3]].copy()
df_chip_exvivo.columns = ['chr','start', 'end', 'value']

In [ ]:
df_chip_exvivo = df_chip_exvivo.sort_values(by=['chr', 'start', 'end'])
df_chip_exvivo = df_chip_exvivo.set_index(['chr'])
df_chip_exvivo.head()

,start,end,value
chr,,,
chr1,0,9950,0.000000
chr1,9950,10000,0.328493
chr1,10000,10050,0.925754
chr1,10050,10100,1.373700
chr1,10100,10150,0.925754


In [ ]:
df_chip_exvivo.isnull().values.any()

False

In [ ]:
from tqdm import tqdm

ls = []

i=0
for row in tqdm(df.itertuples()):
  c=row.chr
  s=row.start
  e=row.end
  T=row.Target
  x119=row.xc119_TPM
  x120=row.xc120_TPM
  cnt=row.cnt
  exp=row.expected
  N=row.Nreads
  sc=row.score
  scwtc=row.score_ATAC_wtcMG
  scchinv=row.score_chip_invitro

  df_sub = df_chip_exvivo.loc[c]
  value = df_sub[(df_sub['start'] <= s) & (df_sub['end'] >= e)]['value']

  if len(value) > 0:
    vv = np.average(value.values)
    value_v = vv
  else:
    value_v = 0.0

  aa = [c,s,e,T,x119,x120,cnt,exp,N,sc,scwtc,scchinv,value_v]
  ls.append(aa)

  i+=1
  if i%5000 == 0:
    print(i)

5070it [00:12, 570.57it/s]

5000


10136it [00:21, 643.77it/s]

10000


15113it [00:28, 774.06it/s]

...

40108it [01:14, 531.84it/s]

40000


45043it [01:24, 463.54it/s]

45000


47730it [01:29, 531.89it/s]


In [ ]:
hm = pd.DataFrame(ls)
hm.columns= ['chr', 'start', 'end', 'Target', 'xc119_TPM', 'xc120_TPM', 'cnt',
             'expected', 'Nreads', 'score', 'score_ATAC_wtcMG','score_chip_invitro','score_chip_exvivo']
hm[hm['score_chip_exvivo'] > 0]

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected,Nreads,score,score_ATAC_wtcMG,score_chip_invitro,score_chip_exvivo
8,chr1,973668,973669,-1.0,20.370000,18.240000,0.0,0.0,6.8,8.670,0.066223,0.000000,0.089589
9,chr1,974180,974181,-1.0,20.370000,18.240000,0.0,0.0,6.8,8.670,0.364224,0.121721,0.059726
10,chr1,974199,974200,-1.0,20.370000,18.240000,0.0,0.0,6.8,8.670,0.364224,0.121721,0.059726
11,chr1,974225,974226,-1.0,20.370000,18.240000,0.0,0.0,6.8,8.670,1.092670,0.000000,0.059726
12,chr1,974296,974297,-1.0,20.370000,18.240000,0.0,0.0,6.8,8.670,1.467930,0.000000,0.089589
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47717,chr9,139922102,139922103,-1.0,3.620000,3.120000,0.0,0.0,10.5,8.980,0.022074,0.395593,0.059726
47720,chr9,140472913,140472914,-1.0,11.220000,8.690000,0.0,0.0,19.5,6.205,0.165556,1.004200,0.836165
47722,chr9,140500443,140500444,-1.0,14.016667,10.953333,0.0,0.0,0.0,0.000,0.474595,0.182581,0.029863
47723,chr9,140506709,140506710,-1.0,14.016667,10.953333,0.0,0.0,0.0,0.000,0.088297,0.273872,0.029863


In [ ]:
df=hm.copy()
del df_chip_exvivo
del hm

# GlassLab2017: ATAC Exvivo Full File

In [ ]:
df_atac_exvivo = pd.read_csv('/content/drive/My Drive/ExvivoMGatac_merge3.bedGraph', header=None,delimiter='\t') 
df_atac_exvivo = df_atac_exvivo.iloc[:,[0,1,2,3]].copy()
df_atac_exvivo.columns = ['chr','start', 'end', 'value']

In [ ]:
df_atac_exvivo = df_atac_exvivo.sort_values(by=['chr', 'start', 'end'])
df_atac_exvivo = df_atac_exvivo.set_index(['chr'])

In [ ]:
df_atac_exvivo.isnull().values.any()

False

In [ ]:
from tqdm import tqdm

ls = []

i=0
for row in tqdm(df.itertuples()):
  c=row.chr
  s=row.start
  e=row.end
  T=row.Target
  x119=row.xc119_TPM
  x120=row.xc120_TPM
  cnt=row.cnt
  exp=row.expected
  N=row.Nreads
  sc=row.score
  scwtc=row.score_ATAC_wtcMG
  scchinv=row.score_chip_invitro
  scchexv=row.score_chip_exvivo

  df_sub = df_atac_exvivo.loc[c]
  value = df_sub[(df_sub['start'] <= s) & (df_sub['end'] >= e)]['value']

  if len(value) > 0:
    vv = np.average(value.values)
    value_v = vv
  else:
    value_v=0.0

  aa = [c,s,e,T,x119,x120,cnt,exp,N,sc,scwtc,scchinv,scchexv,value_v]
  ls.append(aa)

  i+=1
  if i%5000 == 0:
    print(i)

5072it [00:18, 445.93it/s]

5000


10128it [00:28, 637.33it/s]

10000


15093it [00:35, 758.44it/s]

...

40050it [01:37, 368.93it/s]

40000


45048it [01:49, 437.38it/s]

45000


47730it [01:55, 414.36it/s]


In [ ]:
hm = pd.DataFrame(ls)
hm.columns= ['chr', 'start', 'end', 'Target', 'xc119_TPM', 'xc120_TPM', 'cnt',
             'expected', 'Nreads', 'score', 'score_ATAC_wtcMG','score_chip_invitro','score_chip_exvivo','score_atac_exvivo']
hm[hm['score_atac_exvivo'] > 0]

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected,Nreads,score,score_ATAC_wtcMG,score_chip_invitro,score_chip_exvivo,score_atac_exvivo
0,chr1,916549,916550,-1.0,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.066223,0.000000,0.000000,0.034954
1,chr1,961827,961828,-1.0,20.370000,18.240000,11.0,2.431481,6.8,8.670,0.110371,0.000000,0.000000,0.069908
2,chr1,963249,963250,-1.0,20.370000,18.240000,11.0,2.431481,6.8,8.670,0.066223,0.030430,0.000000,0.034954
3,chr1,971224,971225,-1.0,20.370000,18.240000,0.0,0.000000,6.8,8.670,0.154519,0.000000,0.000000,0.139816
5,chr1,973336,973337,-1.0,20.370000,18.240000,0.0,0.000000,6.8,8.670,0.033111,0.000000,0.000000,0.174770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47717,chr9,139922102,139922103,-1.0,3.620000,3.120000,0.0,0.000000,10.5,8.980,0.022074,0.395593,0.059726,0.209724
47720,chr9,140472913,140472914,-1.0,11.220000,8.690000,0.0,0.000000,19.5,6.205,0.165556,1.004200,0.836165,0.069908
47722,chr9,140500443,140500444,-1.0,14.016667,10.953333,0.0,0.000000,0.0,0.000,0.474595,0.182581,0.029863,0.559264
47723,chr9,140506709,140506710,-1.0,14.016667,10.953333,0.0,0.000000,0.0,0.000,0.088297,0.273872,0.029863,0.034954


In [ ]:
df=hm.copy()
del df_atac_exvivo
del hm

# GlassLab2017: ATAC Invitro: Full File

In [ ]:
df_atac_invitro = pd.read_csv('/content/drive/My Drive/invitroMGatac_merge2.bedGraph', header=None,delimiter='\t') 
df_atac_invitro = df_atac_invitro.iloc[:,[0,1,2,3]].copy()
df_atac_invitro.columns = ['chr','start', 'end', 'value']

In [ ]:
df_atac_invitro = df_atac_invitro.sort_values(by=['chr', 'start', 'end'])
df_atac_invitro = df_atac_invitro.set_index(['chr'])

In [ ]:
df_atac_invitro.isnull().values.any()

False

In [ ]:
from tqdm import tqdm

ls = []

i=0
for row in tqdm(df.itertuples()):
  c=row.chr
  s=row.start
  e=row.end
  T=row.Target
  x119=row.xc119_TPM
  x120=row.xc120_TPM
  cnt=row.cnt
  exp=row.expected
  N=row.Nreads
  sc=row.score
  scwtc=row.score_ATAC_wtcMG
  scchinv=row.score_chip_invitro
  scchexv=row.score_chip_exvivo
  scatexv=row.score_atac_exvivo

  df_sub = df_atac_invitro.loc[c]
  value = df_sub[(df_sub['start'] <= s) & (df_sub['end'] >= e)]['value']

  if len(value) > 0:
    vv = np.average(value.values)
    value_v = vv
  else:
    value_v = 0.0

  aa = [c,s,e,T,x119,x120,cnt,exp,N,sc,scwtc,scchinv,scchexv,scatexv,value_v]
  ls.append(aa)

  i+=1
  if i%5000 == 0:
    print(i)

5090it [00:19, 457.97it/s]

5000


10100it [00:31, 649.67it/s]

10000


15106it [00:39, 618.97it/s]

...

40041it [01:45, 372.07it/s]

40000


45056it [02:00, 419.40it/s]

45000


47730it [02:06, 377.86it/s]


In [ ]:
hm = pd.DataFrame(ls)
hm.columns= ['chr', 'start', 'end', 'Target', 'xc119_TPM', 'xc120_TPM', 'cnt',
             'expected', 'Nreads', 'score', 'score_ATAC_wtcMG','score_chip_invitro','score_chip_exvivo','score_atac_exvivo','score_atac_invitro']
hm[hm['score_atac_invitro'] > 0]

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected,Nreads,score,score_ATAC_wtcMG,score_chip_invitro,score_chip_exvivo,score_atac_exvivo,score_atac_invitro
1,chr1,961827,961828,-1.0,20.370000,18.240000,11.0,2.431481,6.8,8.67,0.110371,0.000000,0.000000,0.069908,0.050722
2,chr1,963249,963250,-1.0,20.370000,18.240000,11.0,2.431481,6.8,8.67,0.066223,0.030430,0.000000,0.034954,0.101443
6,chr1,973377,973378,-1.0,20.370000,18.240000,0.0,0.000000,6.8,8.67,0.055185,0.000000,0.000000,0.104862,0.050722
7,chr1,973458,973459,-1.0,20.370000,18.240000,0.0,0.000000,6.8,8.67,0.066223,0.000000,0.000000,0.139816,0.050722
8,chr1,973668,973669,-1.0,20.370000,18.240000,0.0,0.000000,6.8,8.67,0.066223,0.000000,0.089589,0.034954,0.050722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47721,chr9,140488857,140488858,-1.0,22.540000,17.180000,0.0,0.000000,7.0,5.06,0.033111,0.000000,0.000000,0.000000,0.050722
47722,chr9,140500443,140500444,-1.0,14.016667,10.953333,0.0,0.000000,0.0,0.00,0.474595,0.182581,0.029863,0.559264,0.101443
47723,chr9,140506709,140506710,-1.0,14.016667,10.953333,0.0,0.000000,0.0,0.00,0.088297,0.273872,0.029863,0.034954,0.253608
47726,chr9,140578044,140578045,-1.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.011037,0.000000,0.000000,0.000000,0.101443


In [ ]:
df=hm.copy()
del df_atac_invitro
del hm

# eQTL - Full File

In [ ]:
df_eQTL = pd.read_csv('/content/drive/My Drive/all_chr_eqtl.txt',header=None)
df_eQTL.columns = ['rsid','chr','start']

In [ ]:
df_eQTL = df_eQTL.sort_values(by=['chr', 'start', 'rsid'])
df_eQTL['new'] = 'chr' + df_eQTL['chr'].astype(str)
df_eQTL.pop('chr')
df_eQTL.columns = ['rsid','start','chr']
df_eQTL.head()

,rsid,start,chr
2358513,rs28544273,815963,chr1
2318892,rs200141114,816108,chr1
2276271,rs143225517,816376,chr1
2367507,rs3094315,817186,chr1
2369609,rs3131972,817341,chr1


In [ ]:
df = pd.merge(df, df_eQTL, on=['chr','start'],how='left')
df['eQTL_flg'] = df['rsid'].notnull().astype("int")

In [ ]:
df.head()

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected,Nreads,score,score_ATAC_wtcMG,score_chip_invitro,score_chip_exvivo,score_atac_exvivo,score_atac_invitro,rsid,eQTL_flg
0,chr1,916549,916550,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.066223,0.00000,0.0,0.034954,0.000000,NaN,0
1,chr1,961827,961828,-1.0,20.37,18.24,11.0,2.431481,6.8,8.67,0.110371,0.00000,0.0,0.069908,0.050722,NaN,0
2,chr1,963249,963250,-1.0,20.37,18.24,11.0,2.431481,6.8,8.67,0.066223,0.03043,0.0,0.034954,0.101443,NaN,0
3,chr1,971224,971225,-1.0,20.37,18.24,0.0,0.000000,6.8,8.67,0.154519,0.00000,0.0,0.139816,0.000000,NaN,0
4,chr1,972134,972135,-1.0,20.37,18.24,0.0,0.000000,6.8,8.67,0.033111,0.03043,0.0,0.000000,0.000000,NaN,0


In [ ]:
df[df['eQTL_flg']==1]

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected,Nreads,score,score_ATAC_wtcMG,score_chip_invitro,score_chip_exvivo,score_atac_exvivo,score_atac_invitro,rsid,eQTL_flg
1057,chr1,70148744,70148745,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,rs476591,1
1096,chr1,83511408,83511409,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.033111,0.0,0.000000,0.034954,0.000000,rs4907142,1
2167,chr1,207408911,207408912,-1.0,0.00,0.00,0.0,0.000000,8.0,5.68,0.022074,0.0,0.119452,0.000000,0.000000,rs17042520,1
2465,chr1,207863164,207863165,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,rs882198,1
2862,chr1,229799545,229799546,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.088297,0.0,0.000000,0.034954,0.000000,rs489174,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46794,chr9,10606468,10606469,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.165556,0.0,0.000000,0.000000,0.000000,rs62535923,1
46983,chr9,28887332,28887333,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.077260,0.0,0.000000,0.000000,0.000000,rs4352930,1
47064,chr9,32207520,32207521,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.011037,0.0,0.029863,0.000000,0.000000,rs10970833,1
47575,chr9,115476413,115476414,-1.0,13.26,17.41,19.0,6.783486,0.0,0.00,0.000000,0.0,0.209041,0.000000,0.050722,rs146162997,1


In [ ]:
(df.groupby(['chr','start'], as_index=False)['start']
   .agg({'count': 'count'})
   .query('count > 1'))

,chr,start,count
41878,chr7,1594141,2


In [ ]:
df = df[~df.duplicated(subset=['chr','start'])].copy()

In [ ]:
df.head()

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected,Nreads,score,score_ATAC_wtcMG,score_chip_invitro,score_chip_exvivo,score_atac_exvivo,score_atac_invitro,rsid,eQTL_flg
0,chr1,916549,916550,-1.0,0.00,0.00,0.0,0.000000,0.0,0.00,0.066223,0.00000,0.0,0.034954,0.000000,NaN,0
1,chr1,961827,961828,-1.0,20.37,18.24,11.0,2.431481,6.8,8.67,0.110371,0.00000,0.0,0.069908,0.050722,NaN,0
2,chr1,963249,963250,-1.0,20.37,18.24,11.0,2.431481,6.8,8.67,0.066223,0.03043,0.0,0.034954,0.101443,NaN,0
3,chr1,971224,971225,-1.0,20.37,18.24,0.0,0.000000,6.8,8.67,0.154519,0.00000,0.0,0.139816,0.000000,NaN,0
4,chr1,972134,972135,-1.0,20.37,18.24,0.0,0.000000,6.8,8.67,0.033111,0.03043,0.0,0.000000,0.000000,NaN,0


# Validate and Write to Google Drive

In [ ]:
df.drop('rsid', axis=1,inplace=True)

In [ ]:
df['Target'] = df['Target'].astype(int)

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47730 entries, 0 to 47730
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   chr                 47730 non-null  object 
 1   start               47730 non-null  int64  
 2   end                 47730 non-null  int64  
 3   Target              47730 non-null  int64  
 4   xc119_TPM           47730 non-null  float64
 5   xc120_TPM           47730 non-null  float64
 6   cnt                 47730 non-null  float64
 7   expected            47730 non-null  float64
 8   Nreads              47730 non-null  float64
 9   score               47730 non-null  float64
 10  score_ATAC_wtcMG    47730 non-null  float64
 11  score_chip_invitro  47730 non-null  float64
 12  score_chip_exvivo   47730 non-null  float64
 13  score_atac_exvivo   47730 non-null  float64
 14  score_atac_invitro  47730 non-null  float64
 15  eQTL_flg            47730 non-null  int64  
dtypes: f

In [ ]:
## Write df to google drive
df.to_csv('/content/drive/My Drive/Regeneron_STS_2023/Model/Input_Data/SSL_PUNU_Model_Input_Final1.txt', sep='\t',index=False)

In [ ]:
df_old = pd.read_csv('/content/drive/My Drive/Regeneron_STS_2023/Model/Input_Data/SSL_PUNU_Model_Input_Final.txt',delimiter='\t',header=0)

In [ ]:
df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            47721, 47722, 47723, 47724, 47725, 47726, 47727, 47728, 47729,
            47730],
           dtype='int64', length=47730)

In [ ]:
df_old.index

RangeIndex(start=0, stop=47730, step=1)

In [ ]:
df.reset_index(inplace=True)
df.index

RangeIndex(start=0, stop=47730, step=1)

In [ ]:
df.drop('index', axis=1,inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47730 entries, 0 to 47729
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   chr                 47730 non-null  object 
 1   start               47730 non-null  int64  
 2   end                 47730 non-null  int64  
 3   Target              47730 non-null  int64  
 4   xc119_TPM           47730 non-null  float64
 5   xc120_TPM           47730 non-null  float64
 6   cnt                 47730 non-null  float64
 7   expected            47730 non-null  float64
 8   Nreads              47730 non-null  float64
 9   score               47730 non-null  float64
 10  score_ATAC_wtcMG    47730 non-null  float64
 11  score_chip_invitro  47730 non-null  float64
 12  score_chip_exvivo   47730 non-null  float64
 13  score_atac_exvivo   47730 non-null  float64
 14  score_atac_invitro  47730 non-null  float64
 15  eQTL_flg            47730 non-null  int64  
dtypes: f

In [ ]:
print(df_old.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47730 entries, 0 to 47729
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   chr                 47730 non-null  object 
 1   start               47730 non-null  int64  
 2   end                 47730 non-null  int64  
 3   Target              47730 non-null  int64  
 4   xc119_TPM           47730 non-null  float64
 5   xc120_TPM           47730 non-null  float64
 6   cnt                 47730 non-null  float64
 7   expected            47730 non-null  float64
 8   Nreads              47730 non-null  float64
 9   score               47730 non-null  float64
 10  score_ATAC_wtcMG    47730 non-null  float64
 11  score_chip_invitro  47730 non-null  float64
 12  score_chip_exvivo   47730 non-null  float64
 13  score_atac_exvivo   47730 non-null  float64
 14  score_atac_invitro  47730 non-null  float64
 15  eQTL_flg            47730 non-null  int64  
dtypes: f

In [ ]:
df.where(df.values!=df_old.values).notna()

In [ ]:
cols = df.columns
check = pd.DataFrame([], columns = cols)
check

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected,Nreads,score,score_ATAC_wtcMG,score_chip_invitro,score_chip_exvivo,score_atac_exvivo,score_atac_invitro,eQTL_flg


In [ ]:
for ix,row in df.iterrows():
    for col in df.columns:
        if df.loc[ix,col] != df_old.loc[ix,col]:
            check.loc[ix,col] = f"from: {df_old.loc[ix,col]} to: {df.loc[ix,col]}"
        else:
            check.loc[ix,col] = ""

In [ ]:
check.sample(10)

,chr,start,end,Target,xc119_TPM,xc120_TPM,cnt,expected,Nreads,score,score_ATAC_wtcMG,score_chip_invitro,score_chip_exvivo,score_atac_exvivo,score_atac_invitro,eQTL_flg
29784,,,,,,,,,,,,,,,,
43967,,,,,from: 14.45 to: 14.450000000000001,,,,,,,,,,,
37945,,,,,,,,,,,,,,,,
7167,,,,,,,,,,,,,,,,
22152,,,,,,,,,,,,,,,,
47176,,,,,,,,,,,,,,,,
19621,,,,,,,,,,,,,,,,
10531,,,,,,,,,,,,,,,,
2477,,,,,,,,,,,,,,,,
20111,,,,,,,,,,,,,,,,
